In [1]:
import logging

import numpy as np
from queue import Queue, LifoQueue
import time
import copy


class Record:
    point = None  
    point_index = 0  
    value = None  


class Sudo:
    def __init__(self, _data):
        
        self.value = np.array([[0] * 9] * 9, dtype=object)  
        self.new_points = Queue()  
        self.record_queue = LifoQueue()  
        self.guess_times = 0  

        
        self.base_points = [[0, 0], [0, 3], [0, 6], [3, 0], [3, 3], [3, 6], [6, 0], [6, 3], [6, 6]]

        
        self.puzzle = np.array(_data).reshape(9, -1)
        for r in range(0, 9):
            for c in range(0, 9):
                if self.puzzle[r, c]:  # if not Zero
                    # numpy default is int32, convert to int
                    self.value[r, c] = int(self.puzzle[r, c])
                    
                    self.new_points.put((r, c))
                    # logger.debug(f'init: answer={self.value[r, c]} at {(r, c)}')
                else:  # if Zero, guess no. is 1-9
                    self.value[r, c] = [1, 2, 3, 4, 5, 6, 7, 8, 9]

   
    def _cut_num(self, point):
        r, c = point
        val = self.value[r, c]

        
        for i, item in enumerate(self.value[r]):
            if isinstance(item, list):
                if item.count(val):
                    item.remove(val)

                    
                    if len(item) == 1:
                        self.new_points.put((r, i))  
                        logger.debug(f'only one in row: answer={self.value[r, i]} at {(r, i)}')
                        self.value[r, i] = item[0]

        
        for i, item in enumerate(self.value[:, c]):
            if isinstance(item, list):
                if item.count(val):
                    item.remove(val)

                    
                    if len(item) == 1:
                        self.new_points.put((i, c))
                        logger.debug(f'only one in col: answer={self.value[i, c]} at {(i, c)}')
                        self.value[i, c] = item[0]

        
        b_r, b_c = map(lambda x: x // 3 * 3, point)  # 九宫格基准点
        for m_r, row in enumerate(self.value[b_r:b_r + 3, b_c:b_c + 3]):
            for m_c, item in enumerate(row):
                if isinstance(item, list):
                    if item.count(val):
                        item.remove(val)

                        
                        if len(item) == 1:
                            r = b_r + m_r
                            c = b_c + m_c
                            self.new_points.put((r, c))
                            logger.debug(f'only one in block: answer={self.value[r, c]} at {(r, c)}')
                            self.value[r, c] = item[0]

    
    def _check_one_possbile(self):
        
        for r in range(0, 9):
            
            values = list(filter(lambda x: isinstance(x, list), self.value[r]))

            for c, item in enumerate(self.value[r]):
                if isinstance(item, list):
                    for value in item:
                        if sum(map(lambda x: x.count(value), values)) == 1:
                            self.value[r, c] = value
                            self.new_points.put((r, c))
                            logger.debug(f'list val is only one in row: answer={self.value[r, c]} at {(r, c)}')
                            return True

       
        for c in range(0, 9):
            values = list(filter(lambda x: isinstance(x, list), self.value[:, c]))

            for r, item in enumerate(self.value[:, c]):
                if isinstance(item, list):
                    for value in item:
                        if sum(map(lambda x: x.count(value), values)) == 1:
                            self.value[r, c] = value
                            self.new_points.put((r, c))
                            logger.debug(f'list val is only one in col: answer={self.value[r, c]} at {(r, c)}')
                            return True

        
        for r, c in self.base_points:
            
            values = list(filter(lambda x: isinstance(x, list), self.value[r:r + 3, c:c + 3].reshape(1, -1)[0]))

            for m_r, row in enumerate(self.value[r:r + 3, c:c + 3]):
                for m_c, item in enumerate(row):
                    if isinstance(item, list):
                        for value in item:
                            if sum(map(lambda x: x.count(value), values)) == 1:
                                self.value[r + m_r, c + m_c] = value
                                self.new_points.put((r + m_r, c + m_c))
                                logger.debug(f'list val is only one in block: answer={self.value[r + m_r, c +m_c]} at '
                                             f'{(r + m_r, c +m_c)}')
                                return True

    
    def _check_implicit(self):
        for b_r, b_c in self.base_points:
            block = self.value[b_r:b_r + 3, b_c:b_c + 3]

            
            _data = block.reshape(1, -1)[0]
            for i in range(1, 10):
                result = map(lambda x: 0 if not isinstance(x[1], list) else x[0] + 1 if x[1].count(i) else 0,
                             enumerate(_data))
                result = list(filter(lambda x: x > 0, result))
                r_count = len(result)

                if r_count in [2, 3]:
                    
                    rows = list(map(lambda x: (x - 1) // 3, result))
                    cols = list(map(lambda x: (x - 1) % 3, result))

                    if len(set(rows)) == 1:
                        
                        result = list(map(lambda x: b_c + (x - 1) % 3, result))
                        row = b_r + rows[0]

                        for col in range(0, 9):
                            if col not in result:
                                item = self.value[row, col]
                                if isinstance(item, list):
                                    if item.count(i):
                                        item.remove(i)

                                        
                                        if len(item) == 1:
                                            self.new_points.put((row, col))
                                            logger.debug(f'block compare row: answer={self.value[row, col]} at {(row, col)}')
                                            self.value[row, col] = item[0]
                                            return True

                    elif len(set(cols)) == 1:
                       
                        result = list(map(lambda x: b_r + (x - 1) // 3, result))
                        col = b_c + cols[0]

                        for row in range(0, 9):
                            if row not in result:
                                item = self.value[row, col]
                                if isinstance(item, list):
                                    if item.count(i):
                                        item.remove(i)

                                        
                                        if len(item) == 1:
                                            self.new_points.put((row, col))
                                            logger.debug(f'block compare col: answer={self.value[row, col]} at {(row, col)}')
                                            self.value[row, col] = item[0]
                                            return True

    
    def sudo_exclude(self):
        implicit_exist = True
        new_point_exist = True

        while implicit_exist:
            while new_point_exist:
             
                while not self.new_points.empty():
                    point = self.new_points.get()  
                    self._cut_num(point)

                
                new_point_exist = self._check_one_possbile()

           
            implicit_exist = self._check_implicit()
            new_point_exist = True

    
    def get_num_count(self):
        return sum(map(lambda x: 1 if isinstance(x, int) else 0, self.value.reshape(1, -1)[0]))

   
    def get_best_point(self):
        best_score = 0
        best_point = (0, 0)

        for r, row in enumerate(self.value):
            for c, item in enumerate(row):
                point_score = self._get_point_score((r, c))
                if best_score < point_score:
                    best_score = point_score
                    best_point = (r, c)
        return best_point

    def _get_point_score(self, point):
       
        r, c = point
        item = self.value[r, c]

        if isinstance(item, list):
            score = 10 - len(item)
            score += sum(map(lambda x: 1 if isinstance(x, int) else 0, self.value[r]))
            score += sum(map(lambda x: 1 if isinstance(x, int) else 0, self.value[:, c]))
            return score
        else:
            return 0

   
    def verify_value(self):
        # 行
        r = 0
        for row in self.value:
            nums = []
            lists = []
            for item in row:
                (lists if isinstance(item, list) else nums).append(item)
            if len(set(nums)) != len(nums):
                # logger.error(f'verify failed. dup in row {r}')
                logger.debug(f'verify failed. dup in row {r}')
                return False  
            if len(list(filter(lambda x: len(x) == 0, lists))):
                return False  
            r += 1

        # 列
        for c in range(0, 9):
            nums = []
            lists = []
            col = self.value[:, c]

            for item in col:
                (lists if isinstance(item, list) else nums).append(item)
            if len(set(nums)) != len(nums):
                logger.debug(f'verify failed. dup in col {c}')
                return False  
            if len(list(filter(lambda x: len(x) == 0, lists))):
                return False  
        # 九宫格
        for b_r, b_c in self.base_points:
            nums = []
            lists = []
            block = self.value[b_r:b_r + 3, b_c:b_c + 3].reshape(1, -1)[0]

            for item in block:
                (lists if isinstance(item, list) else nums).append(item)
            if len(set(nums)) != len(nums):
                logger.debug(f'verify failed. dup in block {b_r, b_c}')
                return False  
            if len(list(filter(lambda x: len(x) == 0, lists))):
                return False 
        return True

    def add_to_queue(self, point, index):
        record = Record()
        record.point = point
        record.point_index = index
        # recorder.value = self.value.copy() #numpy的copy不行
        record.value = copy.deepcopy(self.value)
        self.record_queue.put(record)
        items = self.value[point]
        self.value[point] = items[index]
        self.new_points.put(point)
        return items

    def sudo_solve_iter(self):
        self.sudo_exclude()
        # logger.debug(f'excluded, current result:\n{self.value}')
        if self.verify_value():
            if self.get_num_count() == 81:
                # solve success
                return
            else:
                logger.info(f'current no. of fixed answers: {self.get_num_count()}')
                point = self.get_best_point()
                index = 0
                items = self.add_to_queue(point, index)
                logger.info(f'add to LIFO queue and guessing {items[index]}/{items}: '
                             f'{[x.point for x in self.record_queue.queue]}')
                self.guess_times += 1
                return self.sudo_solve_iter()
        while True:
            if self.record_queue.empty():
                logger.error(f'Guessed {self.guess_times} times. Sudo is wrong, no answer!')
                exit()
            record = self.record_queue.get()
            point = record.point
            index = record.point_index + 1
            items = record.value[point]
            self.value = record.value
            logger.info(f'Recall! Pop previous point, {items} @{point}')
            if index < len(items):
                items = self.add_to_queue(point, index)
                logger.info(f'guessing next index: answer={items[index]}/{items} @{point}')
                self.guess_times += 1
                return self.sudo_solve_iter()


if __name__ == '__main__':
    
    data = [[8, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 3, 6, 0, 0, 0, 0, 0,
             0, 7, 0, 0, 9, 0, 2, 0, 0,
             0, 5, 0, 0, 0, 7, 0, 0, 0,
             0, 0, 0, 0, 4, 5, 7, 0, 0,
             0, 0, 0, 1, 0, 0, 0, 3, 0,
             0, 0, 1, 0, 0, 0, 0, 6, 8,
             0, 0, 8, 5, 0, 0, 0, 1, 0,
             0, 9, 0, 0, 0, 0, 4, 0, 0],
            [0, 0, 0, 0, 5, 0, 2, 0, 0,
             0, 9, 0, 0, 0, 0, 0, 4, 0,
             0, 0, 0, 0, 1, 0, 0, 0, 0,
             0, 0, 0, 4, 0, 6, 0, 8, 0,
             0, 0, 7, 0, 0, 0, 1, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 8, 0, 9, 4,
             2, 0, 1, 0, 7, 0, 0, 0, 0,
             0, 0, 5, 0, 0, 0, 0, 0, 0]]
    # DEBUG INFO WARNING ERROR CRITICAL
    logging.basicConfig(level=logging.WARN,
                        format='%(asctime)s %(levelname)s %(message)s')
                        # format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)-7s %(message)s')
    logger = logging.getLogger(__name__)
    # try:
    t1 = time.time()
    puzzle = data[0]
    sudo = Sudo(puzzle)
    sudo.sudo_solve_iter()

    logger.warning(f'Done! guessed {sudo.guess_times} times, in {time.time() - t1:.3f}sec')
    logger.warning(f'Puzzle:\n{sudo.puzzle}\nAnswer:\n{sudo.value}')



2018-10-09 20:30:34,797 WARNING Done! guessed 109 times, in 0.324sec
2018-10-09 20:30:34,803 WARNING Puzzle:
[[8 0 0 0 0 0 0 0 0]
 [0 0 3 6 0 0 0 0 0]
 [0 7 0 0 9 0 2 0 0]
 [0 5 0 0 0 7 0 0 0]
 [0 0 0 0 4 5 7 0 0]
 [0 0 0 1 0 0 0 3 0]
 [0 0 1 0 0 0 0 6 8]
 [0 0 8 5 0 0 0 1 0]
 [0 9 0 0 0 0 4 0 0]]
Answer:
[[8 1 2 7 5 3 6 4 9]
 [9 4 3 6 8 2 1 7 5]
 [6 7 5 4 9 1 2 8 3]
 [1 5 4 2 3 7 8 9 6]
 [3 6 9 8 4 5 7 2 1]
 [2 8 7 1 6 9 5 3 4]
 [5 2 1 9 7 4 3 6 8]
 [4 3 8 5 2 6 9 1 7]
 [7 9 6 3 1 8 4 5 2]]
